In [1]:
import segmentation_models_pytorch as smp
from trainer import Trainer

In [2]:
MODEL = 'FPN'
ENCODER = 'efficientnet-b3'
OPTIMIZER = 'Ranger'         # options currently available : Over9000, Ralamb, RAdam, Ranger, LookaheadAdam
LOSS = 'BCE'

In [3]:
if(MODEL == 'Unet'):
    model = smp.Unet(ENCODER, encoder_weights='imagenet', classes=5, activation=None)
elif(MODEL == 'FPN'):
    model = smp.FPN(ENCODER, encoder_weights='imagenet', classes=5, activation=None)
elif(MODEL == 'Linknet'):
    model = smp.Linknet(ENCODER, encoder_weights='imagenet', classes=5, activation=None)

In [ ]:
model_trainer = Trainer(model = model, optim = OPTIMIZER, loss = LOSS, lr = 1e-3, bs = 8, name = ENCODER+'_'+MODEL+'_'+LOSS)

In [ ]:
model_trainer.seed_everything(43)

In [ ]:
model_trainer.change_loader(crop_type=0, shape=512)

In [ ]:
model_trainer.freeze()
model_trainer.fit(2)

Starting epoch: 0 | phase: train | ⏰: 07:35:22


In [ ]:
model_trainer.do_cutmix = False

In [ ]:
model_trainer.unfreeze()
model_trainer.fit(2)

In [ ]:
# def TRAIN(MODEL, ENCODER, OPTIMIZER, LOSS):
#     if(MODEL == 'Unet'):
#         model = smp.Unet(ENCODER, encoder_weights='imagenet', classes=5, activation=None)
#     elif(MODEL == 'FPN'):
#         model = smp.FPN(ENCODER, encoder_weights='imagenet', classes=5, activation=None)
#     elif(MODEL == 'Linknet'):
#         model = smp.Linknet(ENCODER, encoder_weights='imagenet', classes=5, activation=None)
    
#     model_trainer = Trainer(model = model, optim = OPTIMIZER, loss = LOSS, lr = 1e-3, bs = 8, name = ENCODER+'_'+MODEL+'_'+LOSS+'_'+OPTIMIZER)
#     model.cutmix = False
#     model_trainer.freeze()
#     model_trainer.fit(10)
#     model.cutmix = True
#     model_trainer.unfreeze()
#     model_trainer.fit(30)

In [ ]:
# # Accordingly append to the list and train multiple models
# # Naming has also been adjusted 
# # ['Ranger','Over9000'] * ['Dice', 'BCE+DICE', 'BCE+DICE+JACCARD']
# Model = ['Unet']
# Encoder = ['efficientnet-b4']
# Optimizer = ['Ranger', 'Over9000']
# Loss = ['Dice', 'BCE+DICE', 'BCE+DICE+JACCARD']
# for model in Model:
#     for encoder in Encoder:
#         for optimizer in Optimizer:
#             for loss in Loss:
#                 TRAIN(model, encoder, optimizer, loss)
                    
